In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/159 Proj Copy')

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/159 Proj Copy'

In [ ]:
!pip install clean-text emoji
!pip install clean-text
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.5 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=12eeb738269211501a6a3b6163b18f86d6c338aae80b8117fb721c932cd8ca6a
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library Imports

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter

from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import operator
import nltk
import math
from scipy.stats import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertModel, BertTokenizer

import torch
import torch.nn as nn
import random

from cleantext import clean
import re
import emoji
from ast import literal_eval

import spacy
from textblob import TextBlob
from scipy.sparse import hstack
from sklearn.preprocessing import normalize
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('opinion_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

# Load and create Splits folder

In [ ]:
adj_data = pd.read_csv('adj-1.txt', sep='\t', header=None, names=['index_id', 'adjudicated', 'Troll Rating','Post'], index_col=False)
adj_data

,index_id,adjudicated,Troll Rating,Post
0,0,adjudicated,2,Title: Some of yall are so backhanded The am...
1,1,adjudicated,5,Title: Dear Fellow UC Berkeley Students I’ve ...
2,2,adjudicated,5,Title: I want to lose my virginity.
3,3,adjudicated,5,Title: UniversityFuck people’s park 😹 Activis...
4,4,adjudicated,1,Title: UniversityEnough is enough. It’s time t...
...,...,...,...,...
495,495,adjudicated,4,Sex Ed at Berkeley. So I am an incoming freshm...
496,496,adjudicated,1,"Has anyone been charged with a ""Minor in Posse..."
497,497,adjudicated,1,Why are general questions downvoted so heavily...
498,498,adjudicated,2,Fuck the high demand major policy. The whole p...


In [ ]:
# Split the data into training (60%), development (20%), and test (20%)
train, temp = train_test_split(adj_data, test_size=0.4, random_state=42)  # First split for 60% training
dev, test = train_test_split(temp, test_size=0.5, random_state=42)   # Second split for 20% dev and 20% test

In [ ]:
# Directory to store the splits
split_dir = '/content/drive/MyDrive/159 Proj Copy/splits'
os.makedirs(split_dir, exist_ok=True)

# Save the splits to files
train.to_csv(os.path.join(split_dir, 'train.txt'), sep='\t', index=False, header=False)
dev.to_csv(os.path.join(split_dir, 'dev.txt'), sep='\t', index=False, header=False)
test.to_csv(os.path.join(split_dir, 'test.txt'), sep='\t', index=False, header=False)

# Outputs the paths to the created files
train_path = os.path.join(split_dir, 'train.txt')
dev_path = os.path.join(split_dir, 'dev.txt')
test_path = os.path.join(split_dir, 'test.txt')

train_path, dev_path, test_path

('/content/drive/MyDrive/159 Proj Copy/splits/train.txt',
 '/content/drive/MyDrive/159 Proj Copy/splits/dev.txt',
 '/content/drive/MyDrive/159 Proj Copy/splits/test.txt')

In [ ]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y

trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

trainX, trainY=load_data(trainingFile)
devX, devY=load_data(devFile)
testX, testY=load_data(testFile)

print(trainX[0], trainY[0])

Why is race absent in WarnMe descriptions? title. doesn't make any logical sense - if you want to protect students its not helpful to know the color of their hoodie.  
 3


# Baseline Models without any changes

## Majority class classifier

In [ ]:
def majority_class(trainY, evalY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority_class=labelCounts.most_common(1)[0][0]

    correct=0.
    for label in evalY:
        if label == majority_class:
            correct+=1
    return majority_class, correct/len(evalY)


mc, mc_devAcc=majority_class(trainY, devY)
_, mc_testAcc=majority_class(trainY, testY)

print("Majority class: %s, dev accuracy: %.3f, test accuracy: %.3f" % (mc, mc_devAcc, mc_testAcc))

Majority class: 1, dev accuracy: 0.450, test accuracy: 0.410


## Logistics Regression

In [ ]:
class Classifier:

    def __init__(self, feature_method, trainX, trainY, devX, devY, testX, testY):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.min_feature_count=2
        self.log_reg = None

        self.trainY=trainY
        self.devY=devY
        self.testY=testY

        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):

        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    # Train model and evaluate on held-out data
    def train(self):
        (D,F) = self.trainX.shape
        best_dev_accuracy=0
        best_model=None
        for C in [0.1, 1, 10, 100]:
            self.log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
            self.log_reg.fit(self.trainX, self.trainY)
            training_accuracy = self.log_reg.score(self.trainX, self.trainY)
            development_accuracy = self.log_reg.score(self.devX, self.devY)
            if development_accuracy > best_dev_accuracy:
                best_dev_accuracy=development_accuracy
                best_model=self.log_reg

#             print("C: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (C, training_accuracy, development_accuracy))

        self.log_reg=best_model


    def test(self):
        return self.log_reg.score(self.testX, self.testY)


    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()


def binary_bow_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        feats[word]=1

    # print(feats)
    return feats

def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)


In [ ]:
def run(trainingFile, devFile, testFile):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)

    simple_classifier = Classifier(binary_bow_featurize, trainX, trainY, devX, devY, testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()


In [ ]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

run(trainingFile, devFile, testFile)

Test accuracy for best dev model: 0.420, 95% CIs: [0.323 0.517]

1	0.260	so
1	0.251	year
1	0.249	for
1	0.221	s
1	0.216	or
1	0.207	!
1	0.196	last
1	0.195	having
1	0.176	at
1	0.175	a

2	0.213	by
2	0.199	,
2	0.188	come
2	0.186	the
2	0.175	*
2	0.164	world
2	0.162	submitted
2	0.155	self.berkeley
2	0.142	because
2	0.136	but

3	0.322	people
3	0.196	me
3	0.194	why
3	0.179	and
3	0.178	get
3	0.175	good
3	0.167	title
3	0.164	other
3	0.162	take
3	0.154	the

4	0.228	over
4	0.177	most
4	0.160	in
4	0.157	n't
4	0.134	such
4	0.130	's
4	0.128	months
4	0.125	due
4	0.122	everyone
4	0.120	those

5	0.225	on
5	0.188	got
5	0.177	my
5	0.172	real
5	0.159	how
5	0.151	can
5	0.138	love
5	0.132	friend
5	0.126	best
5	0.123	!



## Ordinal Regression

In [ ]:
def load_ordinal_data(filename, ordering):
    X = []
    Y = []
    orig_Y=[]
    for ordinal in ordering:
        Y.append([])

    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)

            index=ordering.index(label)
            for i in range(len(ordering)):
                if index > i:
                    Y[i].append(1)
                else:
                    Y[i].append(0)
            orig_Y.append(label)

    return X, Y, orig_Y

In [ ]:
class OrdinalClassifier:

    def __init__(self, ordinal_values, feature_method, trainX, trainY, devX, devY, testX, testY, orig_trainY, orig_devY, orig_testY):
        self.ordinal_values=ordinal_values
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.min_feature_count=2
        self.log_regs = [None]* (len(self.ordinal_values)-1)

        self.trainY=trainY
        self.devY=devY
        self.testY=testY

        self.orig_trainY=orig_trainY
        self.orig_devY=orig_devY
        self.orig_testY=orig_testY

        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):

        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    def train(self):
        (D,F) = self.trainX.shape


        for idx, ordinal_value in enumerate(self.ordinal_values[:-1]):
            best_dev_accuracy=0
            best_model=None
            for C in [0.1, 1, 10, 100]:

                log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
                log_reg.fit(self.trainX, self.trainY[idx])
                development_accuracy = log_reg.score(self.devX, self.devY[idx])
                if development_accuracy > best_dev_accuracy:
                    best_dev_accuracy=development_accuracy
                    best_model=log_reg


            self.log_regs[idx]=best_model

    def test(self):
        cor=tot=0
        counts=Counter()
        preds=[None]*(len(self.ordinal_values)-1)
        for idx, ordinal_value in enumerate(self.ordinal_values[:-1]):
            preds[idx]=self.log_regs[idx].predict_proba(self.testX)[:,1]

        preds=np.array(preds)

        for data_point in range(len(preds[0])):


            ordinal_preds=np.zeros(len(self.ordinal_values))
            for ordinal in range(len(self.ordinal_values)-1):
                if ordinal == 0:
                    ordinal_preds[ordinal]=1-preds[ordinal][data_point]
                else:
                    ordinal_preds[ordinal]=preds[ordinal-1][data_point]-preds[ordinal][data_point]

            ordinal_preds[len(self.ordinal_values)-1]=preds[len(preds)-1][data_point]

            prediction=np.argmax(ordinal_preds)
            counts[prediction]+=1
            if prediction == self.ordinal_values.index(self.orig_testY[data_point]):
                cor+=1
            tot+=1

        return cor/tot


def binary_bow_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        feats[word]=1

    return feats

def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)


In [ ]:
def run(trainingFile, devFile, testFile, ordinal_values):


    trainX, trainY, orig_trainY=load_ordinal_data(trainingFile, ordinal_values)
    devX, devY, orig_devY=load_ordinal_data(devFile, ordinal_values)
    testX, testY, orig_testY=load_ordinal_data(testFile, ordinal_values)

    simple_classifier = OrdinalClassifier(ordinal_values, binary_bow_featurize, trainX, trainY, devX, devY, testX, testY, orig_trainY, orig_devY, orig_testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY[0]), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

In [ ]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

ordinal_values=["1", "2", "3", "4", "5"]

run(trainingFile, devFile, testFile, ordinal_values)

Test accuracy for best dev model: 0.400, 95% CIs: [0.304 0.496]



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def test_and_evaluate(classifier, testX, testY):
    predictions = classifier.log_reg.predict(testX)

    test_accuracy = classifier.log_reg.score(testX, testY)

    conf_matrix = confusion_matrix(testY, predictions)

    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classifier.log_reg.classes_, yticklabels=classifier.log_reg.classes_)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    return test_accuracy, conf_matrix

simple_classifier = Classifier(binary_bow_featurize, trainX, trainY, devX, devY, testX, testY)
simple_classifier.train()
test_accuracy, conf_matrix = test_and_evaluate(simple_classifier, simple_classifier.testX, simple_classifier.testY)

print("Test Accuracy: {:.3f}".format(test_accuracy))


NameError: name 'Classifier' is not defined

## BERT Classifier

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cpu


In [ ]:
def read_labels(filename):
    labels={}
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[2]
            if label not in labels:
                labels[label]=len(labels)
    return labels

def read_data(filename, labels, max_data_points=1000):

    data = []
    data_labels = []
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[2]
            text = cols[3]

            data.append(text)
            data_labels.append(labels[label])


    # shuffle the data
    tmp = list(zip(data, data_labels))
    random.shuffle(tmp)
    data, data_labels = zip(*tmp)

    if max_data_points is None:
        return data, data_labels

    return data[:max_data_points], data_labels[:max_data_points]

In [ ]:
directory='/content/drive/MyDrive/159 Proj Copy/splits'

In [ ]:
labels=read_labels("%s/train.txt" % directory)
train_x, train_y=read_data("%s/train.txt" % directory, labels, max_data_points=None)
dev_x, dev_y=read_data("%s/dev.txt" % directory, labels, max_data_points=None)
test_x, test_y=read_data("%s/test.txt" % directory, labels, max_data_points=None)

In [ ]:

def evaluate(model, x, y):
    model.eval()
    corr = 0.
    total = 0.
    with torch.no_grad():
        for x, y in zip(x, y):
            y_preds=model.forward(x)
            for idx, y_pred in enumerate(y_preds):
                prediction=torch.argmax(y_pred)
                if prediction == y[idx]:
                    corr += 1.
                total+=1
    return corr/total, total

In [ ]:
class BERTClassifier(nn.Module):

    def __init__(self, bert_model_name, params):
        super().__init__()

        self.model_name=bert_model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=params["doLowerCase"], do_basic_tokenize=False)
        self.bert = BertModel.from_pretrained(self.model_name)

        self.num_labels = params["label_length"]

        self.fc = nn.Linear(params["embedding_size"], self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=16, max_toks=510):

        """ Get batches for input x, y data, with data tokenized according to the BERT tokenizer
      (and limited to a maximum number of WordPiece tokens """

        batches_x=[]
        batches_y=[]

        for i in range(0, len(all_x), batch_size):

            current_batch=[]

            x=all_x[i:i+batch_size]

            batch_x = self.tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=max_toks)
            batch_y=all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))

        return batches_x, batches_y


    def forward(self, batch_x):

        bert_output = self.bert(input_ids=batch_x["input_ids"],
                         attention_mask=batch_x["attention_mask"],
                         token_type_ids=batch_x["token_type_ids"],
                         output_hidden_states=True)

      # We're going to represent an entire document just by its [CLS] embedding (at position 0)
      # And use the *last* layer output (layer -1)
      # as a result of this choice, this embedding will be optimized for this purpose during the training process.

        bert_hidden_states = bert_output['hidden_states']

        out = bert_hidden_states[-1][:,0,:]

        out = self.fc(out)

        return out.squeeze()

def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)


def train_bert(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=None):

    bert_model = BERTClassifier(bert_model_name, params={"label_length": len(labels), "doLowerCase":doLowerCase, "embedding_size":embedding_size})
    bert_model.to(device)

    batch_x, batch_y = bert_model.get_batches(train_x, train_y)
    dev_batch_x, dev_batch_y = bert_model.get_batches(dev_x, dev_y)

    optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)
    cross_entropy=nn.CrossEntropyLoss()

    num_epochs=30
    best_dev_acc = 0.
    patience=5

    best_epoch=0

    for epoch in range(num_epochs):
        bert_model.train()

        # Train
        for x, y in zip(batch_x, batch_y):
            y_pred = bert_model.forward(x)
            loss = cross_entropy(y_pred.view(-1, bert_model.num_labels), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate
        dev_accuracy, _=evaluate(bert_model, dev_batch_x, dev_batch_y)
        if epoch % 1 == 0:
            print("Epoch %s, dev accuracy: %.3f" % (epoch, dev_accuracy))
            if dev_accuracy > best_dev_acc:
                torch.save(bert_model.state_dict(), model_filename)
                best_dev_acc = dev_accuracy
                best_epoch=epoch
        if epoch - best_epoch > patience:
            print("No improvement in dev accuracy over %s epochs; stopping training" % patience)
            break

    bert_model.load_state_dict(torch.load(model_filename))
    print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))
    return bert_model

In [ ]:
bert_model_name="bert-base-cased"
model_filename="mybert.model"
embedding_size=768
doLowerCase=False

model=train_bert(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

test_batch_x, test_batch_y = model.get_batches(test_x, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

# Problems

## Problems we can see here:

1. Classes are imbalance, should explore ways to balance out classes by the following methods

*  Manually resample by repetition of existing minority class -> argument - I don't see the benefit for this because  Repetition can be using text augmentation and give more diversity with the same post but different words.

*   Text augmentation - replace words wtih synonym replacement, random insertions, swaps, and deletion. Help with recognizing copy pasta post.


2. preprocess text data because data models are recognizing stop words and url as label predictions in Logistics regression

*   lowercase
*   remove latin-1, ascii & hex characters
*   extract mentions
*   stop word removal
*   remove URL
*   turn emojis to word description
*   remove self.berkeley and submitted by X year/hours ago
*   replace emails, numbers, url
*   Some unicode issues with quotations - definitely should use Reddit API instead of manually copy and paste reddit post in the future.


3. Feature engineering techniques:

*   repetitive word counts, (detecting copypasta) the higher means more likely to be troll post - achieve through text augmentation

*   bad word list using opinion lexicon to detect sentiment, the higher means more likely to be troll post - using opinion lexicon
*   emoji convert to text
*   basic bag of word count











In [ ]:
import nlpaug.augmenter.word as naw

text_augmenter = naw.SynonymAug(aug_src='wordnet')

# Example: Augmenting a single text string
original_text = "The quick brown fox jumps over the lazy dog"
augmented_text = text_augmenter.augment(original_text)
print("Original:", original_text)
print("Augmented:", augmented_text)

Original: The quick brown fox jumps over the lazy dog
Augmented: ['The quick brown university george fox jumps concluded the lazy dog']


In [ ]:
print(train['Troll Rating'].value_counts())

Troll Rating
1    141
2     62
3     42
5     39
4     16
Name: count, dtype: int64


In [ ]:
# Set all seeds for consistency
random.seed(42)
np.random.seed(42)
english_stopwords = stopwords.words('english')

# Initialize the synonym augmenter
text_augmenter = naw.SynonymAug(aug_src='wordnet', aug_p = random.uniform(0.3, 0.7), stopwords = english_stopwords)

def augment_text(df, class_label, target_count, augmenter):
    class_data = df[df['Troll Rating'] == class_label]
    augmented_texts = []

    needed = target_count - len(class_data)

    while len(augmented_texts) < needed:
        text_to_augment = class_data.sample(1, random_state=np.random.randint(low=1, high=10000))['Post'].values[0]
        augmented_text = augmenter.augment(text_to_augment)

        if isinstance(augmented_text, list):
            augmented_text = ' '.join(augmented_text)

        augmented_texts.append(augmented_text)

    new_data = pd.DataFrame({
        'index_id': random.sample(range(501, 1064), needed),
        'adjudicated': 'adjudicated',
        'Post': augmented_texts,
        'Troll Rating': class_label
    })

    return pd.concat([df, new_data], ignore_index=True)

# Apply augmentation
train_augmented = train.copy()

max_size = train['Troll Rating'].value_counts().max()


for rating in [2, 3, 4, 5]:
    train_augmented = augment_text(train_augmented, rating, max_size, text_augmenter)

# Check the new value counts
print(train_augmented['Troll Rating'].value_counts())

Troll Rating
3    141
5    141
2    141
1    141
4    141
Name: count, dtype: int64


In [ ]:
train_augmented.sort_values('index_id')

,index_id,adjudicated,Troll Rating,Post
220,1,adjudicated,5,Title: Dear Fellow UC Berkeley Students I’ve ...
152,4,adjudicated,1,Title: UniversityEnough is enough. It’s time t...
33,6,adjudicated,2,"Seriously asking, do the protesters blocking t..."
182,8,adjudicated,1,1 month ago by dewpydoodledooCS On Sunday ni...
43,10,adjudicated,4,PSA to any non-Cal student lurking this subred...
...,...,...,...,...
423,1058,adjudicated,3,What happened to all the Israel / Canaan mater...
613,1059,adjudicated,5,Did not tell him he had a big ding a ling (ego...
675,1062,adjudicated,5,Solution to homeless. Hello everybody 1 am an ...
336,1062,adjudicated,2,"About the recently circulating video, an apolo..."


In [ ]:
# Save the splits to files
train_augmented.to_csv(os.path.join(split_dir, 'train_augmented.txt'), sep='\t', index=False, header=False)

# Outputs the paths to the created files
train_path = os.path.join(split_dir, 'train_augmented.txt')

train_path

'/content/drive/MyDrive/159 Proj Copy/splits/train_augmented.txt'

## Data Pre-Processing, Featurizing, Testing

In [ ]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y

text, label = load_data(train_path)

In [ ]:
stop_words = set(stopwords.words('english'))


def clean_custom_text(text):
    # Convert emojis to text
    text = text.replace('self.berkeley', '')

    text = emoji.demojize(text, delimiters=(" ", " "))

    # Use clean-text to perform standard cleaning
    text = clean(text,
                 fix_unicode=False,
                 to_ascii=False,
                 lower=True,                     # lowercase text -> may lose sentiment
                 no_line_breaks=False,           # do not strip line breaks as opposed to only normalizing them
                 no_urls=True,                   # replace all URLs with a special token
                 no_emails=True,                 # replace all email addresses with a special token
                 no_phone_numbers=True,          # replace all phone numbers with a special token
                 no_numbers=False,               # do not replace all numbers with a special token
                 no_digits=False,
                 no_currency_symbols=False,
                 no_punct=False,                 # do not remove punctuations to preserve sentiment
                 replace_with_url="<URL>",
                 replace_with_email="<EMAIL>",
                 replace_with_phone_number="<PHONE>",
                 replace_with_number="<NUMBER>",
                 replace_with_currency_symbol="<CUR>")

    text = text.replace("\\'", "")

    # Regular expression to remove subreddit reference and metadata
    text = re.sub(r'\(.*?self\.berkeley.*?\)', '', text, flags=re.IGNORECASE)

    # Regular expression to remove "submitted X years/months/days/hours ago"
    text = re.sub(r'submitted\s+\d+\s+(years?|months?|days?|hours?)\s+ago', '', text, flags=re.IGNORECASE)

    # Regular expression to remove "by username"
    text = re.sub(r'\s+by\s+\w+', '', text, flags=re.IGNORECASE)

    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha])

    # Further clean up to remove any stray tokens and ensure proper spacing
    text = ' '.join(lemmatized_text.strip().split())

    return text


cleaned_posts = [clean_custom_text(post) for post in text]

In [ ]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y

trainingFile = "splits/train_augmented.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

trainX, trainY=load_data(trainingFile)
devX, devY=load_data(devFile)
testX, testY=load_data(testFile)

print(trainX[0], trainY[0])

Why is race absent in WarnMe descriptions? title. doesn't make any logical sense - if you want to protect students its not helpful to know the color of their hoodie.  
 3


In [ ]:
cleaned_trainX = [clean_custom_text(post) for post in trainX]
cleaned_devX = [clean_custom_text(post) for post in devX]
cleaned_testX = [clean_custom_text(post) for post in testX]

In [ ]:
print(cleaned_trainX[0],cleaned_devX[0], cleaned_testX[0])

race absent warnme description title make logical sense want protect student helpful know color hoodie undecided voter see title I cal student undecided would like hear side I inform junior club nothing put resume I junior school I feel like transfer student I bother get involve campus since everyone talk toxic club culture I put resume since I anything really put I data science major take I could put final project course I would probably look like everyone else school since everyone project edit I mean diss transfer student I think similarly also would thing like club put resume since community college offer much term club people tend less involved since transfer admission much less base extracurricular activity


In [ ]:
# Load opinion lexicon
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.corpus import opinion_lexicon
from nltk.sentiment import SentimentIntensityAnalyzer

def bow_featurize(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Build basic BOW features
    bow_features = {}
    for token in tokens:
        if token in bow_features:
            bow_features[token] += 1
        else:
            bow_features[token] = 1

    # Compute sentiment score using TextBlob
    sentiment_score = TextBlob(text).sentiment.polarity
    bow_features['sentiment_score'] = sentiment_score

    # Compute opinion lexicon counts
    pos_count = sum(1 for word in tokens if word in positive_words)
    neg_count = sum(1 for word in tokens if word in negative_words)

    bow_features['positive_count'] = pos_count
    bow_features['negative_count'] = neg_count

    return bow_features

## Logistics Regression with Cleaning + New Features

In [ ]:
def run(trainingFile, devFile, testFile):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)

    cleaned_trainX = [clean_custom_text(post) for post in trainX]
    cleaned_devX = [clean_custom_text(post) for post in devX]
    cleaned_testX = [clean_custom_text(post) for post in testX]

    simple_classifier = Classifier(bow_featurize, cleaned_trainX, trainY, cleaned_devX, devY, cleaned_testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()



trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

run(trainingFile, devFile, testFile)

Test accuracy for best dev model: 0.390, 95% CIs: [0.294 0.486]

1	0.328	year
1	0.249	last
1	0.219	well
1	0.213	friend
1	0.208	hello
1	0.203	thank
1	0.189	science
1	0.183	haas
1	0.178	phd
1	0.178	check

2	0.234	post
2	0.200	ucla
2	0.178	think
2	0.168	world
2	0.159	come
2	0.144	socal
2	0.141	club
2	0.139	matter
2	0.136	final
2	0.133	u

3	0.287	people
3	0.229	get
3	0.221	gsi
3	0.216	title
3	0.203	know
3	0.201	food
3	0.181	edit
3	0.181	please
3	0.168	use
3	0.161	smoke

4	0.151	cheap
4	0.148	cal
4	0.148	right
4	0.136	survive
4	0.129	enough
4	0.123	tear
4	0.115	everyone
4	0.112	due
4	0.107	push
4	0.107	sex

5	0.246	squirrel
5	0.224	bear
5	0.203	try
5	0.195	one
5	0.185	say
5	0.184	real
5	0.182	get
5	0.175	size
5	0.165	eec
5	0.164	cs



## Logistics Regression with Cleaning + New Features + Text Augmentation w/ Fixed Class imbalance

In [ ]:
def run(trainingFile, devFile, testFile):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)

    cleaned_trainX = [clean_custom_text(post) for post in trainX]
    cleaned_devX = [clean_custom_text(post) for post in devX]
    cleaned_testX = [clean_custom_text(post) for post in testX]

    simple_classifier = Classifier(bow_featurize, cleaned_trainX, trainY, cleaned_devX, devY, cleaned_testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()



trainingFile = "splits/train_augmented.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

run(trainingFile, devFile, testFile)

Test accuracy for best dev model: 0.380, 95% CIs: [0.285 0.475]

1	0.273	last
1	0.250	year
1	0.249	well
1	0.240	hello
1	0.237	thank
1	0.231	friend
1	0.230	science
1	0.216	phd
1	0.205	fact
1	0.202	sub

2	0.328	post
2	0.264	think
2	0.246	evaluation
2	0.235	ucla
2	0.217	final
2	0.213	money
2	0.213	norcal
2	0.207	socal
2	0.206	club
2	0.204	lair

3	0.392	gsi
3	0.373	people
3	0.283	good
3	0.279	edit
3	0.278	title
3	0.269	two
3	0.268	get
3	0.261	bring
3	0.243	food
3	0.231	protect

4	0.477	quantitative
4	0.473	finance
4	0.421	tard
4	0.371	ego
4	0.368	tear
4	0.353	fifth
4	0.295	moffit
4	0.288	cheap
4	0.271	ago
4	0.253	survive

5	0.337	hotbox
5	0.310	squirrel
5	0.307	anal
5	0.282	calbearmatch
5	0.274	campus
5	0.256	try
5	0.254	cholula
5	0.254	sauce
5	0.253	get
5	0.248	press



## Ordinal Regression with Cleaning + New Features

In [ ]:
def run(trainingFile, devFile, testFile, ordinal_values):


    trainX, trainY, orig_trainY=load_ordinal_data(trainingFile, ordinal_values)
    devX, devY, orig_devY=load_ordinal_data(devFile, ordinal_values)
    testX, testY, orig_testY=load_ordinal_data(testFile, ordinal_values)

    cleaned_trainX = [clean_custom_text(post) for post in trainX]
    cleaned_devX = [clean_custom_text(post) for post in devX]
    cleaned_testX = [clean_custom_text(post) for post in testX]


    simple_classifier = OrdinalClassifier(ordinal_values, bow_featurize, cleaned_trainX, trainY, cleaned_devX, devY, cleaned_testX, testY, orig_trainY, orig_devY, orig_testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY[0]), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

In [ ]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

ordinal_values=["1", "2", "3", "4", "5"]

run(trainingFile, devFile, testFile, ordinal_values)

Test accuracy for best dev model: 0.380, 95% CIs: [0.285 0.475]



## Ordinal Regression with Cleaning + New Features + Text Augumentation

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def run(trainingFile, devFile, testFile, ordinal_values):


    trainX, trainY, orig_trainY=load_ordinal_data(trainingFile, ordinal_values)
    devX, devY, orig_devY=load_ordinal_data(devFile, ordinal_values)
    testX, testY, orig_testY=load_ordinal_data(testFile, ordinal_values)

    cleaned_trainX = [clean_custom_text(post) for post in trainX]
    cleaned_devX = [clean_custom_text(post) for post in devX]
    cleaned_testX = [clean_custom_text(post) for post in testX]


    simple_classifier = OrdinalClassifier(ordinal_values, bow_featurize, cleaned_trainX, trainY, cleaned_devX, devY, cleaned_testX, testY, orig_trainY, orig_devY, orig_testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()

    lower, upper=confidence_intervals(accuracy, len(testY[0]), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

In [ ]:
trainingFile = "splits/train_augmented.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

ordinal_values=["1", "2", "3", "4", "5"]

run(trainingFile, devFile, testFile, ordinal_values)

Test accuracy for best dev model: 0.290, 95% CIs: [0.201 0.379]



## BERT with Cleaning + New Features

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


In [ ]:
directory='/content/drive/MyDrive/159 Proj Copy/splits'

In [ ]:
labels=read_labels("%s/train.txt" % directory)
train_x, train_y=read_data("%s/train.txt" % directory, labels, max_data_points=None)
dev_x, dev_y=read_data("%s/dev.txt" % directory, labels, max_data_points=None)
test_x, test_y=read_data("%s/test.txt" % directory, labels, max_data_points=None)

cleaned_trainX = [clean_custom_text(post) for post in train_x]
cleaned_devX = [clean_custom_text(post) for post in dev_x]
cleaned_testX = [clean_custom_text(post) for post in testX]

In [ ]:
# Set all seeds for consistency
random.seed(42)
np.random.seed(42)

bert_model_name="bert-base-cased"
model_filename="mybert.model"
embedding_size=768
doLowerCase=False

model=train_bert(bert_model_name, model_filename, cleaned_trainX, train_y, cleaned_devX, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

test_batch_x, test_batch_y = model.get_batches(cleaned_testX, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

Epoch 0, dev accuracy: 0.450
Epoch 1, dev accuracy: 0.450
Epoch 2, dev accuracy: 0.450
Epoch 3, dev accuracy: 0.440
Epoch 4, dev accuracy: 0.420
Epoch 5, dev accuracy: 0.450
Epoch 6, dev accuracy: 0.440
No improvement in dev accuracy over 5 epochs; stopping training

Best Performing Model achieves dev accuracy of : 0.450
Test accuracy for best dev model: 0.410, 95% CIs: [0.314 0.506]



## BERT with Text Augumentation w/Fixed class imbalance

In [ ]:
labels=read_labels("%s/train_augmented.txt" % directory)
train_x, train_y=read_data("%s/train_augmented.txt" % directory, labels, max_data_points=None)
dev_x, dev_y=read_data("%s/dev.txt" % directory, labels, max_data_points=None)
test_x, test_y=read_data("%s/test.txt" % directory, labels, max_data_points=None)

# cleaned_trainX = [clean_custom_text(post) for post in train_x]
# cleaned_devX = [clean_custom_text(post) for post in dev_x]
# cleaned_testX = [clean_custom_text(post) for post in testX]

In [ ]:
# Set all seeds for consistency
random.seed(42)
np.random.seed(42)

bert_model_name="bert-base-cased"
model_filename="mybert.model"
embedding_size=768
doLowerCase=False

model=train_bert(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

test_batch_x, test_batch_y = model.get_batches(test_x, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

Epoch 0, dev accuracy: 0.340
Epoch 1, dev accuracy: 0.420
Epoch 2, dev accuracy: 0.420
Epoch 3, dev accuracy: 0.460
Epoch 4, dev accuracy: 0.470
Epoch 5, dev accuracy: 0.490
Epoch 6, dev accuracy: 0.500
Epoch 7, dev accuracy: 0.450
Epoch 8, dev accuracy: 0.460
Epoch 9, dev accuracy: 0.440
Epoch 10, dev accuracy: 0.430
Epoch 11, dev accuracy: 0.430
Epoch 12, dev accuracy: 0.430
No improvement in dev accuracy over 5 epochs; stopping training

Best Performing Model achieves dev accuracy of : 0.500
Test accuracy for best dev model: 0.440, 95% CIs: [0.343 0.537]



## BERT with Cleaning + Text Augumentation w/Fixed class imbalance

In [ ]:
labels=read_labels("%s/train_augmented.txt" % directory)
train_x, train_y=read_data("%s/train_augmented.txt" % directory, labels, max_data_points=None)
dev_x, dev_y=read_data("%s/dev.txt" % directory, labels, max_data_points=None)
test_x, test_y=read_data("%s/test.txt" % directory, labels, max_data_points=None)

cleaned_trainX = [clean_custom_text(post) for post in train_x]
cleaned_devX = [clean_custom_text(post) for post in dev_x]
cleaned_testX = [clean_custom_text(post) for post in testX]

NameError: name 'read_labels' is not defined

In [ ]:
# Set all seeds for consistency
random.seed(42)
np.random.seed(42)

bert_model_name="bert-base-cased"
model_filename="mybert.model"
embedding_size=768
doLowerCase=False

model=train_bert(bert_model_name, model_filename, cleaned_trainX, train_y, cleaned_devX, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

test_batch_x, test_batch_y = model.get_batches(cleaned_testX, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

Epoch 0, dev accuracy: 0.140
Epoch 1, dev accuracy: 0.350
Epoch 2, dev accuracy: 0.350
Epoch 3, dev accuracy: 0.420
Epoch 4, dev accuracy: 0.420
Epoch 5, dev accuracy: 0.440
Epoch 6, dev accuracy: 0.410
Epoch 7, dev accuracy: 0.440
Epoch 8, dev accuracy: 0.420
Epoch 9, dev accuracy: 0.440
Epoch 10, dev accuracy: 0.440
Epoch 11, dev accuracy: 0.440
No improvement in dev accuracy over 5 epochs; stopping training

Best Performing Model achieves dev accuracy of : 0.440
Test accuracy for best dev model: 0.350, 95% CIs: [0.257 0.443]



# Analysis

# Ordinal Logistic Regression Confusion Matrix


From our confusion matrix we can see that our model performs the best at predicitng labels with 1 with 30 instances correctly predicted. This may be due to the abduance of 1 labels in our data which makes sense for the model to better at predicitng them. Something seen is that posts labeled are often represented as 1 instead. This problem may be more systematic with our model and deifnitons. Our models may just heavily perfer to label a post as 1 rather than 2 due to lack of 2 labels, the defintions between 1 and 2 may also be too closely linked making it harder to differeniate each other. Our model in general struggles to predict labels 4 in and 5. This may just be due to the pure subtely used in the trolling posts, which may be leading it to becoming more confused. Our model may represent some overfitting to the data which is represented by this confusion matrix. This is further seen with certain key words leading to big impacts in our models prediciton score.

### Does your model learn features of the phenomenon that you didn't consider in your guidelines that might cause you to rethink the category boundaries?

No, I think the model has been coherent with our guidelines base on the result of weights in logistics regression. However, I believe models that uses attention mechanism perform much better than regression methods because of the context that the model understand. This is also the reason why augumentation help improve test accuracy in BERT modeling.

###  What features are learned to most define the classes?

Looking at initial logistics regression weights without any procesing, my model was learning stop words, url links and other metadata, quotations. Therefore, I thought applying text cleaning may help the model benefit from the words. After applying text cleaning, text augmentation, and featurizing. The test accuracy dropped, however, the weights of declaring the labels seem to match our guidelines.

For labels that are "1" - it was "phd" "thank" "friend", etc which I think it often relates to serious post because serious post often show appreciateness and academic oriented - one of the goals the berkeley reddit forum is for.

For labels that are "2" - "ucla", "money", "socal", "norcal" - these show a level of seriousness because it may relates to serious events like deciding between ucla and cal (go bears!), and geographic location differences and adaption. But these can turned into trolling such as jokes around ucla and the cultural differences of socal and norcal.

For labels that are "3" - medium trolling - "gsi" this is often misleading as there were multiple stories of serious post of falling in love with their gsi and trolling posts of copy pasta of squirrels falling in love with their gsi.

For labels that "4" - medium to trolling - I think this may not be enough data to label this class after oversampling. There was only 16 post so oversampling this will cause "quantitivate" "finance" to have heavy weights.

For labels that are "5" - trolling post - "squirrel", "anal", "hotbox" was the top 3 weights. I think this maybe a good representation because squirrel is often used to replace with the person and turned into copy pasta or people come up with squirrel jokes. However, there was a trend of posting squirrel photos before that would label as serious post. Our guide also had sexualizing others and sexual contents. Additionally, the use of slang words often represent trolling post.



### Are there any biases your model makes? (E.g., by performing worse on different dialects or registers of the language

I think the bias I saw was due to low sample size for the labels. I only have 16 labels for "4" medium to trolling posts, after cleaning and augumentation the logistics model instead of recognizing the content, recognize words that appear often in label "4" to predict "4". This is exhibited in our ordinal logistic regression model with more emphasis on overfitting to common words woth 4 rating. A solution to this will be gather more rating "4" labels with diversify posts on Reddit. A possible solution would be a script that scowers a wider range of reddit posts in order to possibly incorporate more 4 ratings to counter this overfitting issue.



### Think about the level of balance in your dataset: Is one label extremely prevalent? How could this impact the model you developed? Is your dataset a good candidate for strategies like oversampling or changing class weights?

Yes, Label "1" as serious post is prevalent being the majority class of 161 counts compared to other classes.

Troll Rating counts
1. 141
2. 62
3. 42
4. 16
5. 39

This can impact the model from not being able to generalize the sentiment and the difference with other classes. I think my dataset is better suited with text arugmentation as oversampling technique to match the majority counts. Thereby making it

Troll Rating counts
1. 141
2. 141
3. 141
4. 141
5. 141

Increase change class weights will lead to another question, how much weights we should assign for each class and how much is fair? - Because we cannot answer this question so my team approach this by text argumentation - randomly changing, considering 30% - 70% of the post and randomly changing words with synonyms to generate new posts.

I did a comparison of each model with 3 techniques: cleaning + featurizing (adding sentiment score, bag of words count, postive + negative word count) + augmentation and cleaning + featurizing, dataset without any changes

Cleaning the text was not beneficial as it loses the tone and words that a person write can be random, misleading so applying text cleaning methods may replace these randomness so this maybe the reason why it loses accuracy for all models.

Text augmentation decreased performance BERT modeling, worst performance in ordinary regression, and decrease in performance for logistics regression. The reason is because our troll post detecting goal, it is not beneficial to featurize as bag of words, rather we need attention mechanism from BERT and masking to understand the whole sentence's rather than ranks and single words like the regression mehtods. Troll post often need to consider sentence context.

#What kind of systematic mistakes does your model make? This could involve reading through test predictions and manually categorizing mistakes that are made

One systematic mistake our model makes is that can Trolling can manifest in various forms that might not always include obvious keywords or repeated patterns. The reliance on bag-of-words models and even the initial BERT models without adequate contextual training may miss subtler forms of trolling expressed through sarcasm, irony, or cultural references. This is seen especially with our ordinal logstic regression model in which our model tends over predict posts as 1 rather than 2, this may be due to the definitons of these buckets being two closley linked to one another. ANother representation of systemic mistakes from the logistic regression model comes from the lack of prediciton on the 4 to 5 once again hinting at the model overfitting to certain key words and not looking for other indicators of trolling, and sarcasm in the posts.